In [53]:
from pytrends.request import TrendReq
import time
import pandas as pd 

In [2]:
pytrends = TrendReq(hl='ko', tz=540)

In [5]:
pytrends.trending_searches(pn='south_korea')

,0
0,폴란드
1,멕시코
2,아르헨티나 사우디아라비아
3,튀니지
4,네덜란드 세네갈
5,양의지
6,STX
7,채은성
8,송민호
9,김건모


In [8]:
with open('text_keywords.txt', 'r') as f:
    keywords = f.read().split('\n')

In [9]:
keywords

['자동차',
 '전자',
 '침구류',
 '청소',
 '실내건축',
 '조리',
 '목재',
 '재생산',
 '제조업',
 '유류',
 '임대업',
 '용접',
 '배전',
 '하수처리',
 '설비',
 '조경',
 '의류',
 '원단',
 '전선',
 '살균',
 '토목',
 '통신',
 '운송업',
 '공공사업',
 '소음방지',
 '기계',
 '수출',
 '수산물',
 '도매업',
 '화장',
 '방수',
 '전기',
 '건설업',
 '재활용',
 '창고',
 '프로그램',
 '도판',
 '철판',
 '염색',
 '가공',
 '서랍',
 '인쇄',
 '출판업',
 '식품',
 '동물',
 '포장',
 '설계용역',
 '도소매',
 '체육',
 '보관',
 '판매업',
 '철근콘크리트',
 '농축산물',
 '건물위생관리',
 '잡화류',
 '의료',
 '난방',
 '아스팔트',
 '인테리어',
 '엔지니어링',
 '음료',
 '무대조명',
 '항체치료제',
 '소방',
 '철구조물',
 '측정장비',
 '가구업',
 '기초무기화합',
 '방송업',
 '온라인',
 '휴대폰',
 '아동복',
 '조미김',
 '점토벽돌',
 '욕탕탕업',
 '호텔숙박업',
 '신발',
 '폐쇄회로',
 '의약품',
 '파견업',
 '골프장',
 '무인감시카메라',
 '김치',
 '유리',
 '학원',
 '레미콘',
 '필름제조',
 '자동화기기',
 '인삼',
 '']

In [16]:
keywords[:10]

['자동차', '전자', '침구류', '청소', '실내건축', '조리', '목재', '재생산', '제조업', '유류']

In [23]:
keywords[5]

'조리'

In [27]:
len(keywords)

90

In [29]:
list(range(5, len(keywords), 5))

[5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85]

In [24]:
pytrends.build_payload(kw_list=['조리'], timeframe='2014-01-01 2022-11-01', geo='KR')
dt = pytrends.interest_over_time()

In [70]:
dataframes = []
j = 1
for i in range(5, len(keywords), 4):
    base_keywords = [keywords[0]]
    base_keywords.extend(keywords[j:i])
    
    pytrends.build_payload(kw_list=base_keywords, timeframe='2014-01-01 2022-11-01', geo='KR')
    dt = pytrends.interest_over_time()
    dt = dt.drop(columns=['isPartial'])
    time.sleep(0.3)
    j = i
    dataframes.append(dt)
if i != len(keywords):
    pytrends.build_payload(kw_list=keywords[i:], timeframe='2014-01-01 2022-11-01', geo='KR')
    dt = pytrends.interest_over_time()
    dt = dt.drop(columns=['isPartial'])
    dataframes.append(dt)

In [95]:
pytrends.build_payload(kw_list=[keywords[0], '한국', 'korea'], timeframe='2014-01-01 2022-11-01', geo='KR')
dt = pytrends.interest_over_time()
dataframes.append(dt.drop(columns=['isPartial']))

In [93]:
df1=dataframes.pop()
df2=dataframes.pop()

In [88]:
keywords[0]

'자동차'

In [94]:
df2.head()

,1,2,3,4,
date,,,,,
2014-01-01,1,0,0,4,87
2014-02-01,1,0,0,4,85
2014-03-01,2,0,0,3,88
2014-04-01,1,0,0,4,78
2014-05-01,1,0,0,4,82


In [96]:
for df in dataframes:
    print(df['자동차'][-1])

49
85
85
85
71
85
85
85
85
85
70
85
85
85
85
85
85
11
85
54
85
85
16


In [105]:
dataframes1 = []
for j, df in enumerate(dataframes):
    years = []
    months = []
    df = df.reset_index(inplace=False)
    for i, row in df.iterrows():
        curr_date = row.date.date()
        years.append(curr_date.year)
        months.append(curr_date.month)
    df = df.drop(columns=['date'])
    if j == 0:
        new_columns = ['year', 'month']
        new_columns.extend(list(df.columns))
        df['year'] = years
        df['month'] = months
        df = df[new_columns]
        base_value = df[keywords[0]].iloc[-1].item()
    else:
        curr_value = df[keywords[0]].iloc[-1].item()
        df = df * base_value / curr_value
        df = df.drop(columns=[keywords[0]])
    dataframes1.append(df)

In [108]:
df_total = pd.concat(dataframes1, axis = 1)
df_total.head()

,year,month,자동차,전자,침구류,청소,실내건축,조리,목재,재생산,...,무인감시카메라,김치,유리,학원,레미콘,필름제조,자동화기기,인삼,한국,korea
0,2014,1,52,52,0,7,0,1.729412,1.729412,0.0,...,0.0,18.447059,16.141176,39.776471,0.576471,0.0,0.0,2.305882,211.3125,116.3750
1,2014,2,47,42,0,8,0,1.729412,1.729412,0.0,...,0.0,15.564706,13.835294,42.658824,0.576471,0.0,0.0,2.305882,214.3750,122.5000
2,2014,3,54,48,0,8,0,2.305882,1.729412,0.0,...,0.0,19.023529,14.411765,35.741176,1.152941,0.0,0.0,1.729412,217.4375,128.6250
3,2014,4,48,47,0,7,0,2.305882,2.305882,0.0,...,0.0,23.635294,12.682353,30.552941,0.576471,0.0,0.0,2.305882,214.3750,143.9375
4,2014,5,48,46,0,9,0,1.729412,1.729412,0.0,...,0.0,23.635294,13.258824,32.858824,0.576471,0.0,0.0,2.305882,205.1875,134.7500


In [115]:
df_total1 = df_total.drop(columns=['기초무기화합', '욕탕탕업'])
df_total1.to_csv('data/google_trends.csv')

In [116]:
df_total1.columns

Index(['year', 'month', '자동차', '전자', '침구류', '청소', '실내건축', '조리', '목재', '재생산',
       '제조업', '유류', '임대업', '용접', '배전', '하수처리', '설비', '조경', '의류', '원단', '전선',
       '살균', '토목', '통신', '운송업', '공공사업', '소음방지', '기계', '수출', '수산물', '도매업', '화장',
       '방수', '전기', '건설업', '재활용', '창고', '프로그램', '도판', '철판', '염색', '가공', '서랍',
       '인쇄', '출판업', '식품', '동물', '포장', '설계용역', '도소매', '체육', '보관', '판매업',
       '철근콘크리트', '농축산물', '건물위생관리', '잡화류', '의료', '난방', '아스팔트', '인테리어', '엔지니어링',
       '음료', '무대조명', '항체치료제', '소방', '철구조물', '측정장비', '가구업', '방송업', '온라인', '휴대폰',
       '아동복', '조미김', '점토벽돌', '호텔숙박업', '신발', '폐쇄회로', '의약품', '파견업', '골프장',
       '무인감시카메라', '김치', '유리', '학원', '레미콘', '필름제조', '자동화기기', '인삼', '한국',
       'korea'],
      dtype='object')

In [113]:
list_google = list(df_total.columns)

In [114]:
list_naver = ['year', 'month', '자동차', '전자', '침구류', '청소', '실내건축', '조리', '목재', '재생산',
       '제조업', '유류', '임대업', '용접', '배전', '하수처리', '설비', '조경', '의류', '원단', '전선',
       '살균', '토목', '통신', '운송업', '공공사업', '소음방지', '기계', '수출', '수산물', '도매업', '화장',
       '방수', '전기', '건설업', '재활용', '창고', '프로그램', '도판', '철판', '염색', '가공', '서랍',
       '인쇄', '출판업', '식품', '동물', '포장', '설계용역', '도소매', '체육', '보관', '판매업',
       '철근콘크리트', '농축산물', '건물위생관리', '잡화류', '의료', '난방', '아스팔트', '인테리어', '엔지니어링',
       '음료', '무대조명', '항체치료제', '소방', '철구조물', '측정장비', '가구업', '방송업', '온라인', '휴대폰',
       '아동복', '조미김', '점토벽돌', '호텔숙박업', '신발', '폐쇄회로', '의약품', '파견업', '골프장',
       '무인감시카메라', '김치', '유리', '학원', '레미콘', '필름제조', '자동화기기', '인삼', '한국']
for keyw in list_google:
    if keyw not in list_naver:
       print(keyw)

기초무기화합
욕탕탕업
korea
